In [3]:
import torch
print(torch.__version__)
print(torch.version.cuda)

1.9.0+cu111
11.1


In [4]:
from typing import Union, Tuple
from torch_geometric.typing import OptPairTensor, Adj, Size # Optional[Tensor], Union[Tensor, SparseTensor], Optional[Tuple[int, int]], all about data type

from torch import Tensor
from torch.nn import Linear
import torch.nn.functional as F
from torch_sparse import SparseTensor, matmul
from torch_geometric.nn.conv import MessagePassing

In [5]:
import warnings
warnings.filterwarnings("ignore")
import os
import time
import random
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from joblib import Parallel, delayed

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

import torch
import torch.nn.functional as F
import torch_geometric.nn as gnn
from torch_geometric.nn import MessagePassing
from torch_geometric.data import Dataset, Data, DataLoader
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from torch_geometric.utils import softmax

In [6]:
inv = 0

In [7]:
df = pd.read_csv('input/CMS_trigger.csv')
print(df.columns)
df

scaler_1 = StandardScaler()
df.loc[:,'Phi_0':'MedianTheta'] = scaler_1.fit_transform(df.loc[:,'Phi_0':'MedianTheta']) # normalize

Index(['Unnamed: 0', 'Phi_0', 'Phi_2', 'Phi_3', 'Phi_4', 'Theta_0', 'Theta_2',
       'Theta_3', 'Theta_4', 'BendingAngle_0', 'BendingAngle_2',
       'BendingAngle_3', 'BendingAngle_4', 'TimeInfo_0', 'TimeInfo_2',
       'TimeInfo_3', 'TimeInfo_4', 'RingNumber_0', 'RingNumber_2',
       'RingNumber_3', 'RingNumber_4', 'Front_0', 'Front_2', 'Front_3',
       'Front_4', 'Mask_0', 'Mask_2', 'Mask_3', 'Mask_4',
       'PatternStraightness', 'Zone', 'MedianTheta', 'q/pt', 'PhiAngle',
       'EtaAngle'],
      dtype='object')


In [8]:
features = ['Phi_'+str(i) for i in [0,2,3,4]] + ['Theta_'+str(i) for i in [0,2,3,4]] + \
['Front_'+str(i) for i in [0,2,3,4]] + ['BendingAngle_'+str(i) for i in [0,2,3,4]] + \
['RingNumber_'+str(i) for i in [0,2,3,4]] + \
['TimeInfo_'+str(i) for i in [0,2,3,4]] + ['Mask_'+str(i) for i in [0,2,3,4]] #+ ['PatternStraightness'] + ['Zone'] + ['MedianTheta']
# edge_index = torch.tensor([(0,1),(1,2),(2,3),(3,2),(2,1),(1,0)], dtype=torch.long).T
edge_index = [(0,1),(1,2),(2,3),(3,2),(2,1),(1,0)]
# edge_index = [(0,1),(1,2),(2,3),(2,4),(1,0),(2,1),(3,2),(4,2)]
# edge_index = [(0,1),(1,2),(2,3),(2,4),(1,0),(0,2),(2,1),(3,2),(4,2),(2,5),(2,6),(5,2),(6,2)]
# edge_index = [(0,1),(1,2),(2,3),(2,4),(1,0),(2,1),(3,2),(4,2),(3,4),(4,3),(1,3),(1,4),(0,3),(0,4),(0,2)]

In [9]:
# x_train, x_test, pT_tr, pT_ts, inv_pT_tr, inv_pT_ts = train_test_split(df[features].to_numpy(), abs(1/df.loc[:,'q/pt']).to_numpy(), 1/abs(1/df.loc[:,'q/pt']).to_numpy(), test_size = 0.2, random_state = 1)
train_mask, test_mask = train_test_split(df['Unnamed: 0'].to_numpy(), test_size = 0.2, random_state = 1)
x_data = df[features].to_numpy()
pT = abs(1/df.loc[:,'q/pt']).to_numpy()
inv_pT = 1/pT
if inv:
    label = inv_pT
else:
    label = pT
num_features = x_data.shape[-1]
print('Data shape: ' + str(x_data.shape))
print(pT.shape)
print('Len train: '+str(len(train_mask))+', Len test: '+str(len(test_mask)))
print('Num. features: '+str(num_features))

Data shape: (1179356, 28)
(1179356,)
Len train: 943484, Len test: 235872
Num. features: 28


In [10]:
def process_data(i):
    
    data = Data(x=torch.tensor(x_data[i].reshape(-1,4).T, dtype=torch.float), y=torch.tensor(label[i], dtype=torch.float),
                edge_index=torch.tensor(edge_index, dtype = torch.long).T)
    return data

In [11]:
class MPL(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(MPL, self).__init__(aggr='add')
        self.mlp1 = torch.nn.Linear(in_channels*2, out_channels)
        self.mlp2 = torch.nn.Linear(in_channels, out_channels)
        self.mlp3 = torch.nn.Linear(2*out_channels, 1)
        self.mlp4 = torch.nn.Linear(2*out_channels, 1)
        self.mlp5 = torch.nn.Linear(in_channels,16)
        self.mlp6 = torch.nn.Linear(out_channels,16)
        self.mlp7 = torch.nn.Linear(16,1)

    def forward(self, x, edge_index):
#         edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        msg = self.propagate(edge_index, x=x)
        x = F.relu(self.mlp2(x))
        w1 = F.sigmoid(self.mlp3(torch.cat([x,msg], dim=1)))
        w2 = F.sigmoid(self.mlp4(torch.cat([x,msg], dim=1)))
        out = w1*msg + w2*x
        
        return out

    def message(self, x_i, x_j, edge_index):
        msg = F.relu(self.mlp1(torch.cat([x_i, x_j-x_i], dim=1)))
        w1 = F.tanh(self.mlp5(x_i))
        w2 = F.tanh(self.mlp6(msg))
        w = self.mlp7(w1*w2)
        w = softmax(w, edge_index[0])
        return msg*w

In [12]:
class MPNN(torch.nn.Module):
    def __init__(self):
        super(MPNN, self).__init__()
        
        # GNN
        self.conv1 = MPL(int(len(features)/4),128)
        self.conv2 = MPL(128,64)
        self.conv3 = MPL(64,64)
        self.conv4 = MPL(64,64)
        self.lin1 = torch.nn.Linear(128+128+128, 128) # gnn + cnn + fcnn
        self.lin2 = torch.nn.Linear(128, 16)
        self.lin3 = torch.nn.Linear(16, 16)
        self.lin4 = torch.nn.Linear(16, 1)
        self.lin5 = torch.nn.Linear(128+128+128, 128) # gnn + cnn + fcnn
        self.lin6 = torch.nn.Linear(128, 16)
        self.lin7 = torch.nn.Linear(16, 16)
        self.lin8 = torch.nn.Linear(16, 1)
        self.global_att_pool1 = gnn.GlobalAttention(torch.nn.Sequential(torch.nn.Linear(64, 1)))
        self.global_att_pool2 = gnn.GlobalAttention(torch.nn.Sequential(torch.nn.Linear(64, 1)))
        
        # CNN
        self.cnv1 = torch.nn.Conv2d(1, 64, (2,2), padding='same')
        self.cnv2 = torch.nn.Conv2d(64, 128, (2,2), padding='same')
        self.cnv3 = torch.nn.Conv2d(128, 256, (2,2), padding='same')
        self.cnv4 = torch.nn.Conv2d(256, 256, (2,2), padding='same')
        self.cnv5 = torch.nn.Conv2d(256, 128, (2,2), padding='same')
        self.cnv6 = torch.nn.Conv2d(128, 128, (2,2), padding='same')
        self.linr1 = torch.nn.Linear(7//2*4//2*128, 256)
        self.drp1 = torch.nn.Dropout(0.5)
        self.linr2 = torch.nn.Linear(256, 128)
        
        # FCNN
        self.linr3 = torch.nn.Linear(len(features), 512)
        self.linr4 = torch.nn.Linear(512, 256)
        self.linr5 = torch.nn.Linear(256, 128)
        self.linr6 = torch.nn.Linear(128, 128)
        self.linr7 = torch.nn.Linear(128, 128)
        
        self.prnt = PrintSize()
    def forward(self, data):
        x_orig, edge_index, batch = data.x, data.edge_index, data.batch
        x_orig2 = torch.reshape(x_orig,(-1,1,4,7))
        x_orig3 = torch.reshape(x_orig,(-1,len(features)))
        
        # gnn
#         x_orig = self.prnt(x_orig)
        x = F.relu(self.conv1(x_orig, edge_index))
        x = F.relu(self.conv2(x, edge_index))
#         x = self.prnt(x) # 8, 64 -- 16, 64 when 4
        x1 = self.global_att_pool1(x, batch)
#         x1 = self.prnt(x1) # 2, 64 --- 4, 64 when 4
        x = F.relu(self.conv3(x, edge_index))
        x = F.relu(self.conv4(x, edge_index))
        x2 = self.global_att_pool2(x, batch)
        x_out = torch.squeeze(torch.cat([x1, x2], dim=-1))
#         x_out = self.prnt(x_out) # 2, 128 -- 4, 128 when 4
        
        # cnn
#         x_orig2 = self.prnt(x_orig2)
        x = F.relu(self.cnv1(x_orig2))
#         x = self.prnt(x)
        x = F.max_pool2d(F.relu(self.cnv2(x)), (2,2))
#         x = self.prnt(x)
        x = F.relu(self.cnv3(x))
        x = F.relu(self.cnv4(x))
        x = F.relu(self.cnv5(x))
        x = F.relu(self.cnv6(x))
#         x = self.prnt(x)
        x = torch.flatten(x, 1)
#         x = self.prnt(x)
        x = F.relu(self.linr1(x))
        x = self.drp1(x)
#         x = self.prnt(x)
        x_out2 = torch.squeeze(F.relu(self.linr2(x)))
#         x_out2 = self.prnt(x_out2)
        
        # fcnn
#         x_orig3 = self.prnt(x_orig3)
        x = F.relu(self.linr3(x_orig3))
#         x = self.prnt(x)
        x = F.relu(self.linr4(x))
        x = F.relu(self.linr5(x))
        x = F.relu(self.linr6(x))
#         x = self.prnt(x)
        x_out3 = F.relu(self.linr7(x))
#         x_out3 = self.prnt(x_out3)
        
        # merge
        x_out4 = torch.cat([x_out, x_out2, x_out3], dim=-1)
#         x_out4 = torch.cat([x_out, x_out3], dim=-1)
#         x_out4 = self.prnt(x_out4)
    
        x = F.relu(self.lin1(x_out4))
        x = F.relu(self.lin2(x))
        x = self.lin3(x)
        xf1 = self.lin4(x).squeeze(-1)
        x = F.relu(self.lin5(x_out4))
        x = F.relu(self.lin6(x))
        x = self.lin7(x)
        xf2 = F.sigmoid(self.lin8(x).squeeze(-1))
        
        return xf1, xf2

In [13]:
class MyDataset(Dataset):
    def __init__(self, indices=list(range(len(df))), transform=None):
        self.transform = transform
        self.indices = indices
    
    def __getitem__(self, idx):
        return process_data(self.indices[idx])
    
    def __len__(self):
        return len(self.indices)

In [14]:
class PrintSize(torch.nn.Module):
    def __init__(self):
        super(PrintSize, self).__init__()
    
    def forward(self, x):
        print(x.shape)
        return x

In [15]:
def multitask_mse(outputs, labels):
    weights = torch.tensor(labels<80, dtype=torch.float).to(device)*labels + torch.tensor(labels>=80, dtype=torch.float).to(device)*torch.tensor(labels<160, dtype=torch.float).to(device)*labels*2.4 + torch.tensor(labels>=160, dtype=torch.float).to(device)*10
    error = weights*(((outputs-labels)/labels)**2)
    return torch.mean(error)

In [16]:
scale = 4e2
mse2 = torch.nn.MSELoss()

In [17]:
def train(prog_bar = True):
    
    train_losses, test_losses = list(), list()
    min_test_loss = float('inf')
    train_loader = DataLoader(MyDataset(indices=train_mask), batch_size=batch_size)
    test_loader = DataLoader(MyDataset(indices=test_mask), batch_size=batch_size)
    
    for epoch in range(epochs):
        train_loss = 0
        test_loss = 0
        if prog_bar:
            pbar = tqdm(train_loader,position=0)
        else:
            pbar = train_loader
            
        # train
        for data in pbar:
            data = data.to(device)
            optimizer.zero_grad()
            outputs1, outputs2 = model(data)
            labels = data.y
            loss = multitask_mse(outputs1, data.y) + scale*mse2(outputs2, 1/data.y)
            loss.backward()
            optimizer.step()
            if prog_bar:
                pbar.set_description('pTLoss: '+str(loss.cpu().detach().numpy()))
                train_loss += loss.cpu().detach().numpy()/len(train_loader)
                
        # test
        for data in test_loader:
            data = data.to(device)
            optimizer.zero_grad()
            outputs1, outputs2 = model(data)
            labels = data.y
            loss = multitask_mse(outputs1, data.y) + scale*mse2(outputs2, 1/data.y)
            test_loss += loss.cpu().detach().numpy()/len(test_loader)
        if test_loss<min_test_loss:
            print('Min loss changed from '+str(min_test_loss)+' to '+str(test_loss))
            min_test_loss = test_loss
            torch.save(model.state_dict(), model_name)
        train_losses.append(train_loss)
        test_losses.append(test_loss)
        if epoch > 10 and min(test_losses[-7:])>min_test_loss+1e-9:
            break
        lr_scheduler.step(test_loss)
        print('Epoch: ', str(epoch+1)+'/'+str(epochs),'| Training pTLoss: ', train_loss, '| Testing pTLoss: ', test_loss)
        
        if not prog_bar:
            plt.plot(train_losses, label="Train Loss")
            plt.plot(test_losses, label="Validation Loss")
            plt.xlabel("# Epoch")
            plt.ylabel("Loss")
            plt.legend(loc='upper right')
            plt.show()
    return train_losses, test_losses

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
batch_size = 2**13
epochs = 50
model = MPNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
# optimizer = Ranger(model.parameters(), lr=0.00005, weight_decay=5e-4) # loss: ~21
# optimizer = RangerLars(model.parameters(), lr=0.01, weight_decay=5e-4)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.002, weight_decay=5e-4)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True, patience=1, factor=0.5)

model_name = 'GNN_v18_Merge_inv_' + str(inv) + '.pth'
train_losses, test_losses = train(prog_bar=True)

  0%|                                                                                          | 0/116 [00:00<?, ?it/s]

Min loss changed from inf to 7.72758439491535
Epoch:  1/50 | Training pTLoss:  50.5363441089104 | Testing pTLoss:  7.72758439491535


  0%|                                                                                          | 0/116 [00:00<?, ?it/s]

Min loss changed from 7.72758439491535 to 3.351206705488007
Epoch:  2/50 | Training pTLoss:  5.6107811866135435 | Testing pTLoss:  3.351206705488007


  0%|                                                                                          | 0/116 [00:00<?, ?it/s]

...

Min loss changed from 2.1783448745464455 to 2.1780728060623695
Epoch:  48/50 | Training pTLoss:  2.0923391025641873 | Testing pTLoss:  2.1780728060623695


  0%|                                                                                          | 0/116 [00:00<?, ?it/s]

Min loss changed from 2.1780728060623695 to 2.1778334831369337
Epoch:  49/50 | Training pTLoss:  2.091837727817996 | Testing pTLoss:  2.1778334831369337


pTLoss: 1.3480669: 100%|█████████████████████████████████████████████████████████████| 116/116 [02:28<00:00,  1.28s/it]


Min loss changed from 2.1778334831369337 to 2.1776351517644414
Epoch:  50/50 | Training pTLoss:  2.0913270126129015 | Testing pTLoss:  2.1776351517644414


In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loaded_model = MPNN().to(device)
rand_model = MPNN().to(device)
loaded_model.load_state_dict(torch.load(model_name))

<All keys matched successfully>

In [22]:
test_loader = DataLoader(MyDataset(indices=test_mask), batch_size=batch_size)
# test
test_los = 0
preds = []
preds2 = []
for data in tqdm(test_loader,position=0):
    data = data.to(device)
    outputs1, outputs2 = model(data)
    labels = data.y
    loss = multitask_mse(outputs1, data.y) + scale*mse2(outputs2, 1/data.y)
    preds.append(outputs1.cpu().detach())
    preds2.append(outputs2.cpu().detach())
    test_los += loss.cpu().detach().numpy()/len(test_loader)
print('Test_loss: '+str(test_los))

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:16<00:00,  1.77it/s]

Test_loss: 2.177634165204804


In [23]:
# save to csv
pred_ls = [float(i) for p in preds for i in p]
pred_ls2 = [float(i) for p in preds2 for i in p]
df_pred = pd.DataFrame(pred_ls)
df_pred2 = pd.DataFrame(pred_ls2)
df_pred.to_csv('GNN_v18_Merge_inv_' + str(0) + '.csv')
df_pred2.to_csv('GNN_v18_Merge_inv_' + str(1) + '.csv')
print('Files saved!')

Files saved!
